In [1]:
%pwd

'/Users/user/Sites/DataScience/MLOps/end-to-end-ml/research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/user/Sites/DataScience/MLOps/end-to-end-ml'

In [5]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/ogunrinde/End-to-end-ML-Project-with-MLflow.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="ogunrinde"
os.environ['MLFLOW_TRACKING_PASSWORD']="0ef668739730a7c7883b6dcc21d9754facb11a8f" 

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    targe_column: str
    mlflow_uri: str

In [31]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            targe_column=schema.name,
            mlflow_uri="https://dagshub.com/ogunrinde/End-to-end-ML-Project-with-MLflow.mlflow"
        )

In [33]:
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import pandas as pd

In [38]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metric(self,actual, pred):
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.targe_column], axis=1)
        test_y = test_data[[self.config.targe_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metric(test_y, predicted_qualities)

            scores = { "rmse": rmse, "mae": mae, "r2": r2 }
            #save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae",mae)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")


In [39]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-13 14:15:03,196: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-13 14:15:03,204: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-13 14:15:03,208: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-13 14:15:03,209: INFO: common: created directory at: artifacts]
[2024-04-13 14:15:03,210: INFO: common: created directory at: artifacts/model_evaluation]


/Users/user/Sites/DataScience/MLOps/end-to-end-ml/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2024/04/13 14:15:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
